In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

from sparknlp.base import DocumentAssembler
from sparknlp.annotator import T5Transformer

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os 
os.environ['PYSPARK_PYTHON'] = "/root/anaconda3/bin/python"     # 设置了运行 PySpark 的 Python 解释器的环境变量

# spark = SparkSession.builder\
#     .config(\
#         "spark.jars",  \
#         "/shareddata/lab09/kafka-clients-3.5.0.jar, \
#         /shareddata/lab09/spark-sql-kafka-0-10_2.12-3.5.0.jar, \
#         /shareddata/lab09/spark-token-provider-kafka-0-10_2.12-3.5.0.jar, \
#         /shareddata/lab09/commons-pool2-2.12.0.jar") \
#     .config("spark.executorEnv.PYSPARK_PYTHON","/root/anaconda3/bin/python")\
#     .config("spark.executor.memory", "16g") \
#     .config("spark.driver.memory", "16g") \
#     .config("spark.log.level", "WARN") \
#     .appName("q1_3").getOrCreate()

# 初始化Spark和Spark NLP
spark = sparknlp.start()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-34be90dd-40b5-4b1b-a63b-c79fd51afce2;1.0
	confs: [default]


KeyboardInterrupt: 

In [ ]:
df = spark.readStream.format("kafka")\
	.option("kafka.bootstrap.servers", "localhost:64050")\
	.option("subscribe", "q1_3")\
	.option("startingOffsets", "earliest")\
    .load()
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
df1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","CAST (timestamp AS TIMESTAMP)")
schema = "input String, output String"
result = df1.selectExpr("key", "from_json(value, '{}') AS data".format(schema), "timestamp")
q3_query = result.writeStream.queryName("QA").format("memory").outputMode("update").trigger(processingTime="5 seconds").start()

24/06/03 22:08:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ccda6736-2a6d-4874-83f8-67eb41a532c6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/03 22:08:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
data_df = spark.sql("SELECT data as text from QA")
data_df.show(5)

+----+
|text|
+----+
+----+



In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

t5 = T5Transformer() \
    .pretrained("/STA323_zc/proj2/flan-t5-small") \
    .setTask("question:")\
    .setMaxOutputLength(200)\
    .setInputCols(["document"]) \
    .setOutputCol("question")

data_df = spark.createDataFrame().toDF("text")

pipeline = Pipeline().setStages([document_assembler, t5])
results = pipeline.fit(data_df).transform(data_df)

results.select("question.result").show(truncate=False)

TypeError: 'JavaPackage' object is not callable

24/06/03 22:08:42 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5968 milliseconds
